In [ ]:
import polars as pl
import matplotlib.pyplot as plt


In [ ]:
df_preproc = pl.read_parquet("data/medicamentos_preprocesados.parquet")

In [ ]:
print("="*80)
print("🔍 ANÁLISIS DE COMPATIBILIDAD PARA HOMOLOGACIÓN")
print("="*80)

# ========================================================================
# 1. SEPARAR VÁLIDOS E INVÁLIDOS
# ========================================================================

mascara_validos = (
    (df_preproc['ESTADO REGISTRO'] == 'Vigente') & 
    (df_preproc['ESTADO CUM'] == 'Activo') & 
    (df_preproc['MUESTRA MÉDICA'] == 'No')
)

validos = df_preproc.filter(mascara_validos)
invalidos = df_preproc.filter(~mascara_validos)

print(f"📊 DATASET BASE:")
print(f"   Total medicamentos: {len(df_preproc):,}")
print(f"   ✅ Válidos (candidatos): {len(validos):,} ({len(validos)/len(df_preproc)*100:.1f}%)")
print(f"   ❌ Inválidos (a homologar): {len(invalidos):,} ({len(invalidos)/len(df_preproc)*100:.1f}%)")


In [ ]:
# ========================================================================
# 2. ANÁLISIS POR PRINCIPIO ACTIVO
# ========================================================================
print(f"\n🧬 ANÁLISIS POR PRINCIPIO ACTIVO:")
print("-" * 60)

# Conjuntos únicos de principios activos
principios_validos = set(validos['PRINCIPIO ACTIVO'].unique().to_list())
principios_invalidos = set(invalidos['PRINCIPIO ACTIVO'].unique().to_list())
principios_comunes = principios_validos.intersection(principios_invalidos)
principios_solo_invalidos = principios_invalidos - principios_validos

print(f"   Principios únicos en válidos: {len(principios_validos):,}")
print(f"   Principios únicos en inválidos: {len(principios_invalidos):,}")
print(f"   Principios comunes (homologables): {len(principios_comunes):,}")
print(f"   Principios solo en inválidos: {len(principios_solo_invalidos):,}")

cobertura_principios = len(principios_comunes) / len(principios_invalidos) * 100
print(f"   🎯 Cobertura por principios: {cobertura_principios:.1f}%")

In [ ]:
# ========================================================================
# 3. ANÁLISIS POR REGISTROS INDIVIDUALES
# ========================================================================

print(f"\n📋 ANÁLISIS POR REGISTROS INDIVIDUALES:")
print("-" * 60)

# Medicamentos inválidos que SÍ pueden homologarse
invalidos_homologables = invalidos.filter(
    pl.col('PRINCIPIO ACTIVO').is_in(list(principios_comunes))
)

# Medicamentos inválidos que NO pueden homologarse
invalidos_no_homologables = invalidos.filter(
    pl.col('PRINCIPIO ACTIVO').is_in(list(principios_solo_invalidos))
)

print(f"   Total inválidos: {len(invalidos):,}")
print(f"   ✅ Homologables: {len(invalidos_homologables):,} ({len(invalidos_homologables)/len(invalidos)*100:.1f}%)")
print(f"   ❌ No homologables: {len(invalidos_no_homologables):,} ({len(invalidos_no_homologables)/len(invalidos)*100:.1f}%)")

cobertura_registros = len(invalidos_homologables) / len(invalidos) * 100
print(f"   🎯 Cobertura por registros: {cobertura_registros:.1f}%")

In [ ]:
# ========================================================================
# 4. ANÁLISIS POR VÍA DE ADMINISTRACIÓN
# ========================================================================

print(f"\n💉 ANÁLISIS POR VÍA DE ADMINISTRACIÓN:")
print("-" * 60)

# Combinaciones (Principio Activo + Vía) disponibles en válidos
combinaciones_validas = validos.select(['PRINCIPIO ACTIVO', 'VÍA ADMINISTRACIÓN']).unique()

# Crear clave combinada para análisis
validos_con_clave = validos.with_columns(
    (pl.col('PRINCIPIO ACTIVO') + '|' + pl.col('VÍA ADMINISTRACIÓN')).alias('CLAVE_COMBO')
)
invalidos_con_clave = invalidos.with_columns(
    (pl.col('PRINCIPIO ACTIVO') + '|' + pl.col('VÍA ADMINISTRACIÓN')).alias('CLAVE_COMBO')
)

# Claves disponibles en válidos
claves_validas = set(validos_con_clave['CLAVE_COMBO'].unique().to_list())

# Medicamentos inválidos que tienen combinación exacta disponible
invalidos_combo_exacto = invalidos_con_clave.filter(
    pl.col('CLAVE_COMBO').is_in(list(claves_validas))
)

print(f"   Combinaciones únicas en válidos: {len(claves_validas):,}")
print(f"   Inválidos con combo exacto disponible: {len(invalidos_combo_exacto):,}")

cobertura_exacta = len(invalidos_combo_exacto) / len(invalidos) * 100
print(f"   🎯 Cobertura exacta (Principio + Vía): {cobertura_exacta:.1f}%")

In [ ]:
# ========================================================================
# 5. ANÁLISIS POR FORMA FARMACÉUTICA
# ========================================================================

print(f"\n💊 ANÁLISIS POR FORMA FARMACÉUTICA:")
print("-" * 60)

# Triple combinación (Principio + Vía + Forma)
validos_triple = validos.with_columns(
    (pl.col('PRINCIPIO ACTIVO') + '|' + 
        pl.col('VÍA ADMINISTRACIÓN') + '|' + 
        pl.col('FORMA FARMACÉUTICA')).alias('CLAVE_TRIPLE')
)
invalidos_triple = invalidos.with_columns(
    (pl.col('PRINCIPIO ACTIVO') + '|' + 
        pl.col('VÍA ADMINISTRACIÓN') + '|' + 
        pl.col('FORMA FARMACÉUTICA')).alias('CLAVE_TRIPLE')
)

claves_triple_validas = set(validos_triple['CLAVE_TRIPLE'].unique().to_list())

invalidos_triple_exacto = invalidos_triple.filter(
    pl.col('CLAVE_TRIPLE').is_in(list(claves_triple_validas))
)

print(f"   Combinaciones triples en válidos: {len(claves_triple_validas):,}")
print(f"   Inválidos con triple exacto: {len(invalidos_triple_exacto):,}")

cobertura_triple = len(invalidos_triple_exacto) / len(invalidos) * 100
print(f"   🎯 Cobertura triple (Principio + Vía + Forma): {cobertura_triple:.1f}%")


In [ ]:
# ========================================================================
# 6. ANÁLISIS DE DENSIDAD DE OPCIONES
# ========================================================================

print(f"\n📈 ANÁLISIS DE DENSIDAD DE OPCIONES:")
print("-" * 60)

# Para cada principio activo homologable, ¿cuántas opciones válidas hay?
opciones_por_principio = validos.group_by('PRINCIPIO ACTIVO').agg([
    pl.len().alias('opciones_disponibles')
]).sort('opciones_disponibles', descending=True)

# Estadísticas de opciones
opciones_stats = opciones_por_principio['opciones_disponibles'].describe()

print(f"   Principios con opciones válidas: {len(opciones_por_principio):,}")
print(f"   Promedio de opciones por principio: {opciones_por_principio['opciones_disponibles'].mean():.1f}")
print(f"   Mediana de opciones: {opciones_por_principio['opciones_disponibles'].median():.1f}")
print(f"   Máximo de opciones: {opciones_por_principio['opciones_disponibles'].max():,}")

# Top 10 principios con más opciones
print(f"\n🏆 TOP 10 PRINCIPIOS CON MÁS OPCIONES VÁLIDAS:")
for i, fila in enumerate(opciones_por_principio.head(10).iter_rows(named=True), 1):
    principio = fila['PRINCIPIO ACTIVO']
    opciones = fila['opciones_disponibles']
    print(f"   {i:2d}. {principio}: {opciones:,} opciones")

# Top 10 principios con menos opciones
print(f"\n📉 TOP 10 PRINCIPIOS CON MENOS OPCIONES VÁLIDAS:")
opciones_menos = opciones_por_principio.sort('opciones_disponibles', descending=False)
for i, fila in enumerate(opciones_menos.head(10).iter_rows(named=True), 1):
    principio = fila['PRINCIPIO ACTIVO']
    opciones = fila['opciones_disponibles']
    print(f"   {i:2d}. {principio}: {opciones:,} opciones")

In [ ]:
# ========================================================================
# 7. RESUMEN EJECUTIVO
# ========================================================================

print(f"\n" + "="*80)
print("🎯 RESUMEN EJECUTIVO DE COMPATIBILIDAD")
print("="*80)

resumen = {
    'total_medicamentos': len(df_preproc),
    'total_validos': len(validos),
    'total_invalidos': len(invalidos),
    'principios_validos': len(principios_validos),
    'principios_invalidos': len(principios_invalidos),
    'principios_comunes': len(principios_comunes),
    'cobertura_principios': cobertura_principios,
    'invalidos_homologables': len(invalidos_homologables),
    'cobertura_registros': cobertura_registros,
    'invalidos_combo_exacto': len(invalidos_combo_exacto),
    'cobertura_exacta': cobertura_exacta,
    'invalidos_triple_exacto': len(invalidos_triple_exacto),
    'cobertura_triple': cobertura_triple,
    'promedio_opciones': opciones_por_principio['opciones_disponibles'].mean()
}

print(f"📊 INDICADORES CLAVE:")
print(f"   🎯 Cobertura por Principio Activo: {resumen['cobertura_registros']:.1f}%")
print(f"   🎯 Cobertura Exacta (PA + Vía): {resumen['cobertura_exacta']:.1f}%") 
print(f"   🎯 Cobertura Triple (PA + Vía + Forma): {resumen['cobertura_triple']:.1f}%")
print(f"   📈 Promedio de opciones por principio: {resumen['promedio_opciones']:.1f}")

# Recomendación estratégica
print(f"\n💡 RECOMENDACIÓN ESTRATÉGICA:")
if resumen['cobertura_registros'] >= 80:
    print("   ✅ EXCELENTE: Alta cobertura. Modelo viable con reglas estrictas.")
elif resumen['cobertura_registros'] >= 60:
    print("   ✅ BUENA: Cobertura aceptable. Modelo viable con algunas flexibilizaciones.")
elif resumen['cobertura_registros'] >= 40:
    print("   ⚠️ MODERADA: Cobertura limitada. Requiere estrategia híbrida.")
else:
    print("   ❌ BAJA: Cobertura insuficiente. Revisar criterios de homologación.")


In [ ]:
# ========================================================================
# 8. VISUALIZACIÓN DE RESULTADOS
# ========================================================================

# Crear visualización de cobertura
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Análisis de Compatibilidad para Homologación', fontsize=16, fontweight='bold')

# 1. Distribución Válidos vs Inválidos
labels = ['Válidos', 'Inválidos']
sizes = [len(validos), len(invalidos)]
colors = ['lightgreen', 'lightcoral']
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax1.set_title('Distribución General')

# 2. Cobertura de Homologación
labels = ['Homologables', 'No Homologables'] 
sizes = [len(invalidos_homologables), len(invalidos_no_homologables)]
colors = ['lightblue', 'lightgray']
ax2.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Medicamentos Inválidos')

# 3. Niveles de Cobertura
coberturas = ['Por Principio', 'Exacta (PA+Vía)', 'Triple (PA+Vía+Forma)']
valores = [cobertura_registros, cobertura_exacta, cobertura_triple]
colors = ['green', 'orange', 'red']
bars = ax3.bar(coberturas, valores, color=colors, alpha=0.7)
ax3.set_title('Niveles de Cobertura (%)')
ax3.set_ylabel('Porcentaje')
ax3.set_ylim(0, 100)

# Agregar valores en las barras
for bar, valor in zip(bars, valores):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 1,
            f'{valor:.1f}%', ha='center', va='bottom')

# 4. Distribución de opciones por principio (Top 20)
top_20_opciones = opciones_por_principio.head(20)
principios_cortos = [p[:20] + '...' if len(p) > 20 else p 
                    for p in top_20_opciones['PRINCIPIO ACTIVO'].to_list()]
opciones_vals = top_20_opciones['opciones_disponibles'].to_list()

ax4.barh(range(len(principios_cortos)), opciones_vals)
ax4.set_yticks(range(len(principios_cortos)))
ax4.set_yticklabels(principios_cortos, fontsize=8)
ax4.set_title('Top 20: Opciones por Principio Activo')
ax4.set_xlabel('Número de opciones válidas')

plt.tight_layout()
plt.show()

print("\n✅ Análisis de compatibilidad completado")

# Guardar resultados en variables
resultados_compatibilidad = resumen
datasets_generados = {
    'validos': validos,
    'invalidos': invalidos,
    'invalidos_homologables': invalidos_homologables,
    'invalidos_no_homologables': invalidos_no_homologables,
    'opciones_por_principio': opciones_por_principio
}

print("\n📋 DATOS DISPONIBLES PARA ANÁLISIS POSTERIORES:")
print(f"   - resumen: {type(resultados_compatibilidad).__name__}")
print(f"   - datasets_generados: {type(datasets_generados).__name__} con {len(datasets_generados)} elementos")

---
atc
---

In [15]:
# ============================================================================
# VERIFICACIÓN: ¿ES EL ATC OBLIGATORIO BASADO EN LOS DATOS REALES?
# ============================================================================

import polars as pl

def verificar_atc_vs_principio_activo(df_preproc):
    """
    Analiza si un mismo principio activo puede tener múltiples códigos ATC,
    lo cual determinaría si ATC debe ser obligatorio u opcional en homologación.
    
    Args:
        df_preproc (pl.DataFrame): DataFrame preprocesado
        
    Returns:
        dict: Análisis de la relación ATC vs Principio Activo
        
    Notes:
        - Si 1 Principio = 1 ATC → ATC es redundante (no obligatorio)
        - Si 1 Principio = múltiples ATC → ATC es crítico (obligatorio)
    """
    
    print("="*80)
    print("🔍 VERIFICACIÓN: RELACIÓN ATC vs PRINCIPIO ACTIVO")
    print("="*80)
    print("📋 Objetivo: Determinar si ATC debe ser obligatorio u opcional")
    
    # ========================================================================
    # 1. ANÁLISIS GENERAL DE LA RELACIÓN
    # ========================================================================
    
    print(f"\n📊 ANÁLISIS GENERAL:")
    print("-" * 60)
    
    # Contar únicos
    total_principios = df_preproc['PRINCIPIO ACTIVO'].n_unique()
    total_atc = df_preproc['ATC'].n_unique()
    total_combinaciones = df_preproc.select(['PRINCIPIO ACTIVO', 'ATC']).unique().shape[0]
    
    print(f"   🧬 Principios activos únicos: {total_principios:,}")
    print(f"   🏷️ Códigos ATC únicos: {total_atc:,}")
    print(f"   🔗 Combinaciones PA-ATC únicas: {total_combinaciones:,}")
    
    # ========================================================================
    # 2. ANÁLISIS DE MAPEO: 1 PRINCIPIO → ¿CUÁNTOS ATC?
    # ========================================================================
    
    print(f"\n🎯 MAPEO: PRINCIPIO ACTIVO → ATC")
    print("-" * 60)
    
    # Para cada principio activo, contar cuántos ATC diferentes tiene
    mapeo_pa_atc = df_preproc.group_by('PRINCIPIO ACTIVO').agg([
        pl.col('ATC').n_unique().alias('atc_diferentes'),
        pl.col('ATC').first().alias('atc_principal'),
        pl.count().alias('total_medicamentos')
    ]).sort('atc_diferentes', descending=True)
    
    # Estadísticas del mapeo
    principios_con_multiples_atc = mapeo_pa_atc.filter(pl.col('atc_diferentes') > 1)
    principios_con_un_atc = mapeo_pa_atc.filter(pl.col('atc_diferentes') == 1)
    
    print(f"   📈 Principios con UN SOLO ATC: {len(principios_con_un_atc):,} ({len(principios_con_un_atc)/len(mapeo_pa_atc)*100:.1f}%)")
    print(f"   📈 Principios con MÚLTIPLES ATC: {len(principios_con_multiples_atc):,} ({len(principios_con_multiples_atc)/len(mapeo_pa_atc)*100:.1f}%)")
    
    if len(principios_con_multiples_atc) > 0:
        max_atc = principios_con_multiples_atc['atc_diferentes'].max()
        print(f"   📈 Máximo ATC por principio: {max_atc}")
        
        print(f"\n🔝 TOP 15 PRINCIPIOS CON MÁS ATC DIFERENTES:")
        for i, fila in enumerate(principios_con_multiples_atc.head(15).iter_rows(named=True), 1):
            principio = fila['PRINCIPIO ACTIVO']
            num_atc = fila['atc_diferentes']
            medicamentos = fila['total_medicamentos']
            print(f"      {i:2d}. {principio[:50]}{'...' if len(principio) > 50 else ''}")
            print(f"          → {num_atc} ATC diferentes, {medicamentos:,} medicamentos")
    
    # ========================================================================
    # 3. ANÁLISIS INVERSO: 1 ATC → ¿CUÁNTOS PRINCIPIOS?
    # ========================================================================
    
    print(f"\n🔄 MAPEO INVERSO: ATC → PRINCIPIO ACTIVO")
    print("-" * 60)
    
    # Para cada ATC, contar cuántos principios activos diferentes tiene
    mapeo_atc_pa = df_preproc.group_by('ATC').agg([
        pl.col('PRINCIPIO ACTIVO').n_unique().alias('principios_diferentes'),
        pl.col('PRINCIPIO ACTIVO').first().alias('principio_principal'),
        pl.count().alias('total_medicamentos')
    ]).sort('principios_diferentes', descending=True)
    
    atc_con_multiples_principios = mapeo_atc_pa.filter(pl.col('principios_diferentes') > 1)
    atc_con_un_principio = mapeo_atc_pa.filter(pl.col('principios_diferentes') == 1)
    
    print(f"   📈 ATC con UN SOLO principio: {len(atc_con_un_principio):,} ({len(atc_con_un_principio)/len(mapeo_atc_pa)*100:.1f}%)")
    print(f"   📈 ATC con MÚLTIPLES principios: {len(atc_con_multiples_principios):,} ({len(atc_con_multiples_principios)/len(mapeo_atc_pa)*100:.1f}%)")
    
    if len(atc_con_multiples_principios) > 0:
        max_principios = atc_con_multiples_principios['principios_diferentes'].max()
        print(f"   📈 Máximo principios por ATC: {max_principios}")
        
        print(f"\n🔝 TOP 10 ATC CON MÁS PRINCIPIOS DIFERENTES:")
        for i, fila in enumerate(atc_con_multiples_principios.head(10).iter_rows(named=True), 1):
            atc = fila['ATC']
            num_principios = fila['principios_diferentes']
            medicamentos = fila['total_medicamentos']
            print(f"      {i:2d}. {atc}: {num_principios} principios, {medicamentos:,} medicamentos")
    
    # ========================================================================
    # 4. EJEMPLOS ESPECÍFICOS DE CONFLICTOS
    # ========================================================================
    
    if len(principios_con_multiples_atc) > 0:
        print(f"\n🔍 EJEMPLOS DE PRINCIPIOS CON MÚLTIPLES ATC:")
        print("-" * 60)
        
        # Tomar los 3 principios con más ATC diferentes
        ejemplos = principios_con_multiples_atc.head(3)
        
        for i, fila in enumerate(ejemplos.iter_rows(named=True), 1):
            principio = fila['PRINCIPIO ACTIVO']
            print(f"\n   📋 EJEMPLO {i}: {principio}")
            
            # Obtener todos los ATC para este principio
            atc_del_principio = df_preproc.filter(
                pl.col('PRINCIPIO ACTIVO') == principio
            ).select(['ATC', 'VÍA ADMINISTRACIÓN', 'FORMA FARMACÉUTICA']).unique().sort('ATC')
            
            print(f"      ATC encontrados:")
            for j, detalle in enumerate(atc_del_principio.iter_rows(named=True), 1):
                atc = detalle['ATC']
                via = detalle['VÍA ADMINISTRACIÓN']
                forma = detalle['FORMA FARMACÉUTICA']
                print(f"         {j}. {atc} | {via} | {forma}")
    
    # ========================================================================
    # 5. IMPACTO EN HOMOLOGACIÓN
    # ========================================================================
    
    print(f"\n📊 IMPACTO EN HOMOLOGACIÓN:")
    print("-" * 60)
    
    # Si solo consideramos PA + VÍA vs PA + VÍA + ATC
    combinaciones_pa_via = df_preproc.select(['PRINCIPIO ACTIVO', 'VÍA ADMINISTRACIÓN']).unique().shape[0]
    combinaciones_pa_via_atc = df_preproc.select(['PRINCIPIO ACTIVO', 'VÍA ADMINISTRACIÓN', 'ATC']).unique().shape[0]
    
    print(f"   🔗 Combinaciones PA + Vía: {combinaciones_pa_via:,}")
    print(f"   🔗 Combinaciones PA + Vía + ATC: {combinaciones_pa_via_atc:,}")
    
    reduccion_combinaciones = ((combinaciones_pa_via_atc - combinaciones_pa_via) / combinaciones_pa_via * 100)
    print(f"   📈 Incremento de especificidad con ATC: {reduccion_combinaciones:.1f}%")
    
    # Medicamentos que se separarían solo por ATC
    medicamentos_separados_por_atc = combinaciones_pa_via_atc - combinaciones_pa_via
    print(f"   🎯 Combinaciones adicionales por ATC: {medicamentos_separados_por_atc:,}")
    
    # ========================================================================
    # 6. CONCLUSIÓN BASADA EN DATOS
    # ========================================================================
    
    print(f"\n" + "="*80)
    print("🎯 CONCLUSIÓN BASADA EN TUS DATOS")
    print("="*80)
    
    # Calcular porcentajes clave
    pct_principios_multiples_atc = len(principios_con_multiples_atc) / len(mapeo_pa_atc) * 100
    pct_atc_multiples_principios = len(atc_con_multiples_principios) / len(mapeo_atc_pa) * 100
    
    # Crear resumen
    resumen = {
        'total_principios': total_principios,
        'total_atc': total_atc,
        'principios_con_multiples_atc': len(principios_con_multiples_atc),
        'pct_principios_multiples_atc': pct_principios_multiples_atc,
        'atc_con_multiples_principios': len(atc_con_multiples_principios),
        'pct_atc_multiples_principios': pct_atc_multiples_principios,
        'combinaciones_pa_via': combinaciones_pa_via,
        'combinaciones_pa_via_atc': combinaciones_pa_via_atc,
        'incremento_especificidad': reduccion_combinaciones
    }
    
    print(f"📊 HALLAZGOS CLAVE:")
    print(f"   🧬 {pct_principios_multiples_atc:.1f}% de principios tienen múltiples ATC")
    print(f"   🏷️ {pct_atc_multiples_principios:.1f}% de ATC tienen múltiples principios")
    print(f"   🎯 ATC añade {reduccion_combinaciones:.1f}% más especificidad")
    
    # Decisión automática basada en datos
    print(f"\n💡 RECOMENDACIÓN BASADA EN DATOS:")
    
    if pct_principios_multiples_atc > 20:
        print("   ✅ ATC DEBE SER OBLIGATORIO")
        print("   📋 Razón: Muchos principios tienen múltiples ATC → Diferenciación importante")
        recomendacion = "OBLIGATORIO"
    elif pct_principios_multiples_atc > 10:
        print("   ⚠️ ATC DEBE SER PREFERENCIAL")
        print("   📋 Razón: Algunos principios tienen múltiples ATC → Usar para ranking")
        recomendacion = "PREFERENCIAL"
    else:
        print("   ❌ ATC PUEDE SER OPCIONAL")
        print("   📋 Razón: Pocos principios tienen múltiples ATC → Redundante con PA")
        recomendacion = "OPCIONAL"
    
    resumen['recomendacion'] = recomendacion
    
    print(f"\n🎯 ESTRATEGIA RECOMENDADA:")
    if recomendacion == "OBLIGATORIO":
        print("   Filtro duro: PRINCIPIO_ACTIVO + VÍA + ATC")
        print("   Ranking: por FORMA_FARMACÉUTICA + CANTIDAD")
    elif recomendacion == "PREFERENCIAL":
        print("   Filtro duro: PRINCIPIO_ACTIVO + VÍA")
        print("   Scoring: bonus por ATC igual + FORMA_FARMACÉUTICA + CANTIDAD")
    else:
        print("   Filtro duro: PRINCIPIO_ACTIVO + VÍA")
        print("   Ranking: por FORMA_FARMACÉUTICA + CANTIDAD")
    
    return resumen, {
        'mapeo_pa_atc': mapeo_pa_atc,
        'mapeo_atc_pa': mapeo_atc_pa,
        'principios_con_multiples_atc': principios_con_multiples_atc,
        'atc_con_multiples_principios': atc_con_multiples_principios
    }

# ============================================================================
# EJECUTAR VERIFICACIÓN
# ============================================================================

resumen_atc, datos_atc = verificar_atc_vs_principio_activo(df_preproc)

print(f"\n📋 Variable creada: resumen_atc['recomendacion'] = '{resumen_atc['recomendacion']}'")

🔍 VERIFICACIÓN: RELACIÓN ATC vs PRINCIPIO ACTIVO
📋 Objetivo: Determinar si ATC debe ser obligatorio u opcional

📊 ANÁLISIS GENERAL:
------------------------------------------------------------
   🧬 Principios activos únicos: 20,321
   🏷️ Códigos ATC únicos: 2,419
   🔗 Combinaciones PA-ATC únicas: 25,120

🎯 MAPEO: PRINCIPIO ACTIVO → ATC
------------------------------------------------------------
   📈 Principios con UN SOLO ATC: 18,324 (90.2%)
   📈 Principios con MÚLTIPLES ATC: 1,997 (9.8%)
   📈 Máximo ATC por principio: 36

🔝 TOP 15 PRINCIPIOS CON MÁS ATC DIFERENTES:
       1. CLORURO DE SODIO
          → 36 ATC diferentes, 2,940 medicamentos
       2. HIDROCLOROTIAZIDA
          → 33 ATC diferentes, 4,361 medicamentos
       3. FENILEFRINA CLORHIDRATO
          → 29 ATC diferentes, 2,772 medicamentos
       4. ACETAMINOFEN
          → 28 ATC diferentes, 4,755 medicamentos
       5. ESTEARATO DE MAGNESIO
          → 27 ATC diferentes, 160 medicamentos
       6. BICARBONATO DE SODIO
   

C:\Users\ORPHEUX\AppData\Local\Temp\ipykernel_13352\894837164.py:55: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('total_medicamentos')
C:\Users\ORPHEUX\AppData\Local\Temp\ipykernel_13352\894837164.py:88: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('total_medicamentos')


In [ ]:
df_preproc

CUM,EXPEDIENTE CUM,CONSECUTIVO,ESTADO REGISTRO,CANTIDAD CUM,ESTADO CUM,MUESTRA MÉDICA,UNIDAD,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,CONCENTRACIÓN,PRINCIPIO ACTIVO,UNIDAD MEDIDA,CANTIDAD,UNIDAD REFERENCIA,FORMA FARMACÉUTICA
str,i64,i64,str,f64,str,str,str,str,str,str,str,str,str,f64,str,str
"""10042-1""",10042,1,"""Vencido""",20.0,"""Inactivo""","""No""","""U""","""N07CA03""","""FLUNARIZINA""","""ORAL""","""A""","""FLUNARICINA""","""mg""",10.0,"""TABLETAS""","""TABLETA"""
"""10045-1""",10045,1,"""Vencido""",10.0,"""Inactivo""","""No""","""U""","""M01AE01""","""IBUPROFENO""","""ORAL""","""A""","""IBUPROFENO""","""mg""",400.0,"""TABLETAS""","""TABLETA"""
"""10045-2""",10045,2,"""Vencido""",30.0,"""Inactivo""","""No""","""U""","""M01AE01""","""IBUPROFENO""","""ORAL""","""A""","""IBUPROFENO""","""mg""",400.0,"""TABLETAS""","""TABLETA"""
"""100454-1""",100454,1,"""Vencido""",6.0,"""Inactivo""","""No""","""U""","""G01AF02""","""CLOTRIMAZOL""","""VAGINAL""","""A""","""CLOTRIMAZOL""","""mg""",100.0,"""TABLETA VAGINAL""","""TABLETA VAGINAL"""
"""100458-1""",100458,1,"""Vencido""",100.0,"""Inactivo""","""No""","""U""","""M01AB05""","""DICLOFENACO""","""ORAL""","""A""","""DICLOFENACO SÓDICO""","""mg""",50.0,"""TABLETA""","""TABLETA CUBIERTA CON PELICULA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""9870-7""",9870,7,"""Vencido""",200.0,"""Inactivo""","""No""","""U""","""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""A""","""CIPROFLOXACINA CLORHIDRATO MON…","""mg""",500.0,"""TABLETA""","""TABLETA CUBIERTA (GRAGEA)"""
"""9870-7""",9870,7,"""Vencido""",200.0,"""Inactivo""","""No""","""U""","""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""A""","""CIPROFLOXACINA CLORHIDRATO MON…","""mg""",500.0,"""TABLETA""","""TABLETA CUBIERTA (GRAGEA)"""
"""9870-8""",9870,8,"""Vencido""",300.0,"""Inactivo""","""No""","""U""","""J01MA02""","""CIPROFLOXACINA""","""ORAL""","""A""","""CIPROFLOXACINA CLORHIDRATO MON…","""mg""",500.0,"""TABLETA""","""TABLETA CUBIERTA (GRAGEA)"""


: 